In [1]:
import cv2, os
import sqlite3
from PIL import Image
import numpy as np

cam = cv2.VideoCapture(0)
detector=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
conn=sqlite3.connect("FaceBase.db")

def insertOrUpdate(Id, Name):
        
    cmd="SELECT * FROM People WHERE ID="+str(Id)
  
    cursor=conn.execute(cmd)
    isRecordExist=0
    for row in cursor:
        isRecordExist=1
    if(isRecordExist==1):
        cmd="UPDATE People SET Name='"+str(Name)+"'WHERE ID="+str(Id)
    else:
        cmd="INSERT INTO People(ID, Name) Values(" + str(Id) + ",'" + str(Name)+"')"
    
    conn.execute(cmd)
    conn.commit()

def getImagesWithID(path):
    curId = 1;
    faces=[]
    Ids=[]
    userPaths=[os.path.join(path, f) for f in os.listdir(path)] 
    
    for userPath in userPaths:
        imagePaths = [os.path.join(userPath, f) for f in os.listdir(userPath)]
        
        for imagePath in imagePaths:
            faceImg = Image.open(imagePath).convert('L')
            faceNp = np.array(faceImg, "uint8")
            
            faces.append(faceNp)
            Ids.append(curId)
            
            cv2.imshow("training", faceNp)
            cv2.waitKey(10)
            
        curId = curId + 1
        
    return Ids, faces    
 

def getUserCount():
    cmd="SELECT * FROM People"
  
    # Connection 으로부터 Cursor 생성
    cur = conn.cursor()
    
    # SQL 쿼리 실행
    cur.execute(cmd)

    # 데이타 Fetch
    rows = cur.fetchall()
    c = len(rows)
    
    return c+1
    
    
name = input('name: ')
Id = str(getUserCount())
os.mkdir('dataSet/User'+ Id)

#id=input("enter your id")
#name=input("enter your name")
insertOrUpdate(getUserCount(),name)
sampleNum = 0
while True:
    ret, im = cam.read()
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    faces=detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    for(x,y,w,h) in faces:
        sampleNum=sampleNum+1
        cv2.imwrite("dataSet/User"+str(Id) +'/'+ str(sampleNum) + ".jpg", gray[y:y+h,x:x+w])
        cv2.rectangle(im, (x-50, y-50), (x+w+50, y+h+50),(225, 0, 0), 2)
    cv2.imshow('im', im)
    cv2.waitKey(100)
    if sampleNum>40:
        break


recognizer = cv2.face.createLBPHFaceRecognizer()

path="dataSet"
Ids,faces = getImagesWithID(path)
recognizer.train(faces, np.array(Ids))
recognizer.save(os.path.join(os.path.curdir, 'recognizer/trained' + '.yml'))

cam.release()
cv2.destroyAllWindows()
conn.close()
           

name: byungdoo


error: D:\Build\OpenCV\opencv-3.2.0\modules\imgproc\src\color.cpp:9748: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor
